# Análisis de Componentes Principales (PCA) y Detección de Outliers
## Datos Hidrológicos Completos

**Objetivo:** Aplicar técnicas de reducción de dimensionalidad (PCA) y detección de valores atípicos a los datos hidrológicos completos.

**Dataset:** `datos_hidrologicos_completos.csv` (1940-2024, ~31,000 registros)

**Metodología:**
1. Análisis Exploratorio de Datos (EDA)
2. Preprocesamiento de datos
3. Análisis de Componentes Principales (PCA)
4. Detección de Outliers (múltiples métodos)
5. Visualización y conclusiones

## 1. Importar Librerías

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Librerías para PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Librerías para detección de outliers
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from scipy import stats

# Configuración de visualización
import matplotlib
# Configurar backend para visualización
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
    print("Usando backend: inline")
except:
    # Si falla, intentar con TkAgg para entornos de escritorio
    try:
        matplotlib.use('TkAgg')
        print("Usando backend: TkAgg")
    except:
        print(f"Usando backend por defecto: {matplotlib.get_backend()}")

# Configurar estilo
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
        print("Usando estilo por defecto")

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas exitosamente")
print(f"✓ Matplotlib backend activo: {matplotlib.get_backend()}")

## 2. Cargar y Explorar Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../data/processed/datos_hidrologicos_completos.csv')
df['fecha'] = pd.to_datetime(df['fecha'])

print(f"Dimensiones del dataset: {df.shape}")
print(f"\nPeriodo de datos: {df['fecha'].min()} a {df['fecha'].max()}")
print(f"Total de días: {len(df)}")
df.head()

In [ ]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
df.info()

In [ ]:
# Estadísticas descriptivas
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 80)
df.describe()

In [ ]:
# Análisis de valores faltantes
print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 80)

missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores Faltantes': missing_values,
    'Porcentaje (%)': missing_percent
}).sort_values(by='Valores Faltantes', ascending=False)

print(missing_df[missing_df['Valores Faltantes'] > 0])

# Visualizar valores faltantes
plt.figure(figsize=(12, 6))
missing_data = missing_df[missing_df['Valores Faltantes'] > 0]
if len(missing_data) > 0:
    plt.barh(missing_data.index, missing_data['Porcentaje (%)'])
    plt.xlabel('Porcentaje de Valores Faltantes (%)')
    plt.title('Valores Faltantes por Variable')
    plt.tight_layout()
    plt.show()
else:
    print("\n¡No hay valores faltantes en el dataset!")

## 3. Preprocesamiento de Datos

In [ ]:
# Seleccionar solo variables numéricas (excluyendo fecha y datos_completos)
# Nota: almacenamiento_hm3 tiene muchos valores faltantes, lo excluiremos del análisis PCA

print("Seleccionando variables numéricas para el análisis...")

# Columnas a excluir
exclude_cols = ['fecha', 'datos_completos', 'almacenamiento_hm3']

# Seleccionar columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"\nVariables seleccionadas para el análisis ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols, 1):
    print(f"{i:2d}. {col}")

# Crear dataset para análisis
df_analysis = df[numeric_cols].copy()

# Verificar si hay valores faltantes
print(f"\nValores faltantes en el dataset de análisis: {df_analysis.isnull().sum().sum()}")

# Si hay valores faltantes, imputar con la mediana
if df_analysis.isnull().sum().sum() > 0:
    print("Imputando valores faltantes con la mediana...")
    df_analysis = df_analysis.fillna(df_analysis.median())
    print("Imputación completada.")

print(f"\nDimensiones del dataset de análisis: {df_analysis.shape}")

In [ ]:
# Visualizar distribuciones de las variables
print("Generando gráficos de distribución...")

n_cols = 4
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 3))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    axes[i].hist(df_analysis[col], bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_title(col, fontsize=9)
    axes[i].set_xlabel('Valor')
    axes[i].set_ylabel('Frecuencia')
    axes[i].grid(True, alpha=0.3)

# Ocultar ejes sobrantes
for i in range(len(numeric_cols), len(axes)):
    axes[i].axis('off')

plt.suptitle('Distribución de Variables Numéricas', fontsize=14, y=1.001)
plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación
print("Calculando matriz de correlación...")

plt.figure(figsize=(14, 12))
correlation_matrix = df_analysis.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables Hidrológicas', fontsize=14, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Identificar correlaciones fuertes
print("\nCorrelaciones fuertes (|r| > 0.7):")
high_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

high_corr_df = pd.DataFrame(high_corr, columns=['Variable 1', 'Variable 2', 'Correlación'])
high_corr_df = high_corr_df.sort_values(by='Correlación', ascending=False, key=abs)
print(high_corr_df.to_string(index=False))

In [ ]:
# Estandarización de datos (requerida para PCA)
print("Estandarizando datos...")

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_analysis)
df_scaled_df = pd.DataFrame(df_scaled, columns=numeric_cols, index=df_analysis.index)

print(f"Datos estandarizados: {df_scaled_df.shape}")
print("\nEstadísticas después de estandarización (deberían tener media ~0 y std ~1):")
print(df_scaled_df.describe().loc[['mean', 'std']])

## 4. Análisis de Componentes Principales (PCA)

In [ ]:
# Aplicar PCA con todas las componentes
print("Aplicando PCA...")

pca_full = PCA()
pca_full.fit(df_scaled)

# Varianza explicada
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"\nNúmero total de componentes: {len(explained_variance)}")
print(f"Varianza explicada por las primeras 5 componentes:")
for i in range(min(5, len(explained_variance))):
    print(f"  PC{i+1}: {explained_variance[i]:.4f} ({explained_variance[i]*100:.2f}%)")
print(f"\nVarianza acumulada con 5 componentes: {cumulative_variance[4]:.4f} ({cumulative_variance[4]*100:.2f}%)")

In [ ]:
# Gráfico de varianza explicada (Scree Plot)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Varianza individual
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Componente Principal')
axes[0].set_ylabel('Varianza Explicada')
axes[0].set_title('Varianza Explicada por Componente (Scree Plot)')
axes[0].grid(True, alpha=0.3)

# Varianza acumulada
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-', linewidth=2)
axes[1].axhline(y=0.80, color='r', linestyle='--', label='80% varianza')
axes[1].axhline(y=0.90, color='g', linestyle='--', label='90% varianza')
axes[1].axhline(y=0.95, color='b', linestyle='--', label='95% varianza')
axes[1].set_xlabel('Número de Componentes')
axes[1].set_ylabel('Varianza Acumulada')
axes[1].set_title('Varianza Explicada Acumulada')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determinar número de componentes para 80%, 90%, 95% de varianza
n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f"\nComponentes necesarias para:")
print(f"  80% de varianza: {n_components_80} componentes")
print(f"  90% de varianza: {n_components_90} componentes")
print(f"  95% de varianza: {n_components_95} componentes")

In [ ]:
# Aplicar PCA con número óptimo de componentes (usaremos 90% de varianza)
optimal_components = n_components_90
print(f"Aplicando PCA con {optimal_components} componentes (captura ~90% de varianza)...")

pca = PCA(n_components=optimal_components)
pca_data = pca.fit_transform(df_scaled)

# Crear DataFrame con componentes principales
pca_columns = [f'PC{i+1}' for i in range(optimal_components)]
df_pca = pd.DataFrame(pca_data, columns=pca_columns, index=df_analysis.index)

# Agregar fecha al DataFrame PCA
df_pca['fecha'] = df['fecha'].values

print(f"\nDimensiones del dataset PCA: {df_pca.shape}")
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum():.4f} ({pca.explained_variance_ratio_.sum()*100:.2f}%)")
df_pca.head()

In [ ]:
# Análisis de loadings (pesos de las variables en cada componente)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loadings_df = pd.DataFrame(
    loadings,
    columns=pca_columns,
    index=numeric_cols
)

print("\nLOADINGS DE LAS PRIMERAS 3 COMPONENTES PRINCIPALES")
print("=" * 80)
print(loadings_df.iloc[:, :3].to_string())

# Visualizar loadings de las primeras 3 componentes
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

for i in range(3):
    pc = f'PC{i+1}'
    loadings_sorted = loadings_df[pc].sort_values(key=abs, ascending=False)
    
    axes[i].barh(range(len(loadings_sorted)), loadings_sorted.values, 
                 color=['red' if x < 0 else 'blue' for x in loadings_sorted.values],
                 alpha=0.7, edgecolor='black')
    axes[i].set_yticks(range(len(loadings_sorted)))
    axes[i].set_yticklabels(loadings_sorted.index, fontsize=8)
    axes[i].set_xlabel('Loading')
    axes[i].set_title(f'{pc} ({pca.explained_variance_ratio_[i]*100:.1f}% varianza)')
    axes[i].axvline(x=0, color='black', linewidth=0.5)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Loadings de las Primeras 3 Componentes Principales', fontsize=14, y=1.001)
plt.tight_layout()
plt.show()

In [ ]:
# Biplot: PC1 vs PC2
fig, ax = plt.subplots(figsize=(14, 10))

# Scatter plot de las observaciones (muestrear para mejor visualización)
sample_size = min(5000, len(df_pca))
sample_indices = np.random.choice(len(df_pca), sample_size, replace=False)
ax.scatter(df_pca.iloc[sample_indices]['PC1'], 
           df_pca.iloc[sample_indices]['PC2'], 
           alpha=0.3, s=10)

# Agregar vectores de loadings
scale_factor = 3  # Factor para visualizar mejor los vectores
for i, var in enumerate(numeric_cols):
    ax.arrow(0, 0, 
             loadings_df.loc[var, 'PC1'] * scale_factor, 
             loadings_df.loc[var, 'PC2'] * scale_factor,
             head_width=0.1, head_length=0.1, fc='red', ec='red', alpha=0.7)
    ax.text(loadings_df.loc[var, 'PC1'] * scale_factor * 1.15, 
            loadings_df.loc[var, 'PC2'] * scale_factor * 1.15,
            var, fontsize=8, ha='center', va='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% varianza)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% varianza)')
ax.set_title('Biplot: PC1 vs PC2', fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización de las primeras 3 componentes principales en 3D
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Muestrear datos para mejor visualización
sample_size = min(5000, len(df_pca))
sample_indices = np.random.choice(len(df_pca), sample_size, replace=False)

scatter = ax.scatter(df_pca.iloc[sample_indices]['PC1'],
                     df_pca.iloc[sample_indices]['PC2'],
                     df_pca.iloc[sample_indices]['PC3'],
                     c=df_pca.iloc[sample_indices].index,
                     cmap='viridis', alpha=0.5, s=10)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)')
ax.set_title('Visualización 3D: PC1, PC2, PC3', fontsize=14)

plt.colorbar(scatter, label='Índice temporal', pad=0.1)
plt.tight_layout()
plt.show()

## 5. Detección de Outliers

### 5.1. Método 1: Z-Score (Distancia de Mahalanobis simplificada)

In [ ]:
# Detección de outliers usando Z-score en datos estandarizados
print("Método 1: Detección de Outliers usando Z-Score")
print("=" * 80)

# Calcular Z-score para cada variable
z_scores = np.abs(stats.zscore(df_analysis))

# Umbral: puntos con Z-score > 3 en cualquier variable
threshold = 3
outliers_zscore = (z_scores > threshold).any(axis=1)

print(f"Umbral Z-score: {threshold}")
print(f"Outliers detectados: {outliers_zscore.sum()} ({outliers_zscore.sum()/len(df)*100:.2f}%)")
print(f"Observaciones normales: {(~outliers_zscore).sum()} ({(~outliers_zscore).sum()/len(df)*100:.2f}%)")

### 5.2. Método 2: Isolation Forest

In [ ]:
# Detección de outliers usando Isolation Forest
print("Método 2: Detección de Outliers usando Isolation Forest")
print("=" * 80)

# Configurar Isolation Forest
iso_forest = IsolationForest(
    contamination=0.05,  # Esperamos ~5% de outliers
    random_state=42,
    n_estimators=100
)

# Aplicar en datos estandarizados
outliers_iso = iso_forest.fit_predict(df_scaled)
outliers_iso_bool = outliers_iso == -1  # -1 indica outlier

# Scores de anomalía (más negativo = más anómalo)
anomaly_scores_iso = iso_forest.score_samples(df_scaled)

print(f"Outliers detectados: {outliers_iso_bool.sum()} ({outliers_iso_bool.sum()/len(df)*100:.2f}%)")
print(f"Observaciones normales: {(~outliers_iso_bool).sum()} ({(~outliers_iso_bool).sum()/len(df)*100:.2f}%)")
print(f"\nScore de anomalía (más negativo = más anómalo):")
print(f"  Mínimo: {anomaly_scores_iso.min():.4f}")
print(f"  Máximo: {anomaly_scores_iso.max():.4f}")
print(f"  Media: {anomaly_scores_iso.mean():.4f}")

### 5.3. Método 3: Local Outlier Factor (LOF)

In [ ]:
# Detección de outliers usando Local Outlier Factor (LOF)
print("Método 3: Detección de Outliers usando Local Outlier Factor (LOF)")
print("=" * 80)

# Configurar LOF
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.05  # Esperamos ~5% de outliers
)

# Aplicar en datos estandarizados
outliers_lof = lof.fit_predict(df_scaled)
outliers_lof_bool = outliers_lof == -1  # -1 indica outlier

# Scores negativos de LOF (más negativo = más anómalo)
lof_scores = lof.negative_outlier_factor_

print(f"Outliers detectados: {outliers_lof_bool.sum()} ({outliers_lof_bool.sum()/len(df)*100:.2f}%)")
print(f"Observaciones normales: {(~outliers_lof_bool).sum()} ({(~outliers_lof_bool).sum()/len(df)*100:.2f}%)")
print(f"\nLOF Score (más negativo = más anómalo):")
print(f"  Mínimo: {lof_scores.min():.4f}")
print(f"  Máximo: {lof_scores.max():.4f}")
print(f"  Media: {lof_scores.mean():.4f}")

### 5.4. Método 4: Elliptic Envelope (Asumiendo distribución Gaussiana multivariada)

In [ ]:
# Detección de outliers usando Elliptic Envelope
print("Método 4: Detección de Outliers usando Elliptic Envelope")
print("=" * 80)

# Configurar Elliptic Envelope
elliptic = EllipticEnvelope(
    contamination=0.05,  # Esperamos ~5% de outliers
    random_state=42
)

# Aplicar en datos estandarizados
outliers_elliptic = elliptic.fit_predict(df_scaled)
outliers_elliptic_bool = outliers_elliptic == -1  # -1 indica outlier

# Scores de Mahalanobis distance
mahalanobis_scores = elliptic.score_samples(df_scaled)

print(f"Outliers detectados: {outliers_elliptic_bool.sum()} ({outliers_elliptic_bool.sum()/len(df)*100:.2f}%)")
print(f"Observaciones normales: {(~outliers_elliptic_bool).sum()} ({(~outliers_elliptic_bool).sum()/len(df)*100:.2f}%)")
print(f"\nMahalanobis Distance Score (más negativo = más anómalo):")
print(f"  Mínimo: {mahalanobis_scores.min():.4f}")
print(f"  Máximo: {mahalanobis_scores.max():.4f}")
print(f"  Media: {mahalanobis_scores.mean():.4f}")

### 5.5. Método 5: Outliers en el espacio PCA

In [ ]:
# Detección de outliers en el espacio PCA usando distancia euclidiana
print("Método 5: Detección de Outliers en el Espacio PCA")
print("=" * 80)

# Calcular distancia euclidiana desde el origen en el espacio PCA
pca_distances = np.sqrt(np.sum(pca_data**2, axis=1))

# Usar percentil 95 como umbral
threshold_pca = np.percentile(pca_distances, 95)
outliers_pca_bool = pca_distances > threshold_pca

print(f"Umbral (percentil 95): {threshold_pca:.4f}")
print(f"Outliers detectados: {outliers_pca_bool.sum()} ({outliers_pca_bool.sum()/len(df)*100:.2f}%)")
print(f"Observaciones normales: {(~outliers_pca_bool).sum()} ({(~outliers_pca_bool).sum()/len(df)*100:.2f}%)")
print(f"\nDistancia PCA:")
print(f"  Mínima: {pca_distances.min():.4f}")
print(f"  Máxima: {pca_distances.max():.4f}")
print(f"  Media: {pca_distances.mean():.4f}")
print(f"  Mediana: {np.median(pca_distances):.4f}")

### 5.6. Comparación de Métodos

In [ ]:
# Crear DataFrame con todos los resultados
outliers_comparison = pd.DataFrame({
    'fecha': df['fecha'],
    'Z-Score': outliers_zscore,
    'Isolation_Forest': outliers_iso_bool,
    'LOF': outliers_lof_bool,
    'Elliptic_Envelope': outliers_elliptic_bool,
    'PCA_Distance': outliers_pca_bool
})

# Agregar columna de consenso (outlier si al menos 3 métodos lo detectan)
outliers_comparison['Consenso_3+'] = (
    outliers_comparison[['Z-Score', 'Isolation_Forest', 'LOF', 'Elliptic_Envelope', 'PCA_Distance']].sum(axis=1) >= 3
)

print("\nCOMPARACIÓN DE MÉTODOS DE DETECCIÓN DE OUTLIERS")
print("=" * 80)
print("\nNúmero de outliers detectados por cada método:")
for col in ['Z-Score', 'Isolation_Forest', 'LOF', 'Elliptic_Envelope', 'PCA_Distance', 'Consenso_3+']:
    count = outliers_comparison[col].sum()
    pct = count / len(df) * 100
    print(f"  {col:25s}: {count:6d} ({pct:5.2f}%)")

In [ ]:
# Matriz de confusión entre métodos
from sklearn.metrics import confusion_matrix

methods = ['Z-Score', 'Isolation_Forest', 'LOF', 'Elliptic_Envelope', 'PCA_Distance']
n_methods = len(methods)

fig, axes = plt.subplots(n_methods, n_methods, figsize=(16, 16))

for i, method1 in enumerate(methods):
    for j, method2 in enumerate(methods):
        if i == j:
            # Diagonal: mostrar distribución del método
            axes[i, j].hist([0, 1], weights=[
                (~outliers_comparison[method1]).sum(),
                outliers_comparison[method1].sum()
            ], bins=2, edgecolor='black', alpha=0.7)
            axes[i, j].set_xticks([0.25, 0.75])
            axes[i, j].set_xticklabels(['Normal', 'Outlier'])
            axes[i, j].set_title(method1, fontsize=10)
        else:
            # Fuera de diagonal: matriz de confusión
            cm = confusion_matrix(outliers_comparison[method1], outliers_comparison[method2])
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[i, j],
                       xticklabels=['Normal', 'Outlier'],
                       yticklabels=['Normal', 'Outlier'])
            if j == 0:
                axes[i, j].set_ylabel(method1, fontsize=9)
            if i == n_methods - 1:
                axes[i, j].set_xlabel(method2, fontsize=9)

plt.suptitle('Comparación entre Métodos de Detección de Outliers', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
# Diagrama de Venn simplificado (usando consenso)
print("\nAnálisis de Consenso:")
print("=" * 80)

# Contar cuántos métodos detectan cada observación como outlier
outlier_counts = outliers_comparison[methods].sum(axis=1)

print("\nDistribución de detecciones:")
for i in range(6):
    count = (outlier_counts == i).sum()
    pct = count / len(df) * 100
    print(f"  Detectado por {i} métodos: {count:6d} ({pct:5.2f}%)")

# Visualizar distribución
plt.figure(figsize=(10, 6))
counts = [((outlier_counts == i).sum()) for i in range(6)]
plt.bar(range(6), counts, edgecolor='black', alpha=0.7)
plt.xlabel('Número de Métodos que Detectan como Outlier')
plt.ylabel('Frecuencia')
plt.title('Consenso entre Métodos de Detección de Outliers')
plt.xticks(range(6))
plt.grid(True, alpha=0.3, axis='y')

# Agregar línea vertical en consenso 3+
plt.axvline(x=2.5, color='r', linestyle='--', linewidth=2, label='Umbral Consenso (3+ métodos)')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Visualización de Outliers

In [ ]:
# Visualizar outliers en el espacio PCA (PC1 vs PC2)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

outlier_methods = [
    ('Z-Score', outliers_zscore),
    ('Isolation_Forest', outliers_iso_bool),
    ('LOF', outliers_lof_bool),
    ('Elliptic_Envelope', outliers_elliptic_bool),
    ('PCA_Distance', outliers_pca_bool),
    ('Consenso_3+', outliers_comparison['Consenso_3+'])
]

for idx, (method_name, outlier_mask) in enumerate(outlier_methods):
    # Plotear normales
    axes[idx].scatter(df_pca.loc[~outlier_mask, 'PC1'], 
                     df_pca.loc[~outlier_mask, 'PC2'],
                     c='blue', s=10, alpha=0.3, label='Normal')
    # Plotear outliers
    axes[idx].scatter(df_pca.loc[outlier_mask, 'PC1'], 
                     df_pca.loc[outlier_mask, 'PC2'],
                     c='red', s=30, alpha=0.7, marker='x', label='Outlier')
    
    axes[idx].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    axes[idx].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    axes[idx].set_title(f'{method_name}\n({outlier_mask.sum()} outliers, {outlier_mask.sum()/len(df)*100:.2f}%)')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Outliers en el Espacio PCA (PC1 vs PC2)', fontsize=14, y=1.001)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización 3D de outliers en espacio PCA
fig = plt.figure(figsize=(16, 12))

for idx, (method_name, outlier_mask) in enumerate(outlier_methods[:4], 1):
    ax = fig.add_subplot(2, 2, idx, projection='3d')
    
    # Normales
    ax.scatter(df_pca.loc[~outlier_mask, 'PC1'],
               df_pca.loc[~outlier_mask, 'PC2'],
               df_pca.loc[~outlier_mask, 'PC3'],
               c='blue', s=5, alpha=0.3, label='Normal')
    
    # Outliers
    ax.scatter(df_pca.loc[outlier_mask, 'PC1'],
               df_pca.loc[outlier_mask, 'PC2'],
               df_pca.loc[outlier_mask, 'PC3'],
               c='red', s=30, alpha=0.8, marker='x', label='Outlier')
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)')
    ax.set_title(f'{method_name}\n({outlier_mask.sum()} outliers)')
    ax.legend()

plt.suptitle('Outliers en el Espacio PCA 3D', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización temporal de outliers (usando consenso)
fig, ax = plt.subplots(figsize=(16, 6))

# Plotear todas las observaciones
ax.scatter(outliers_comparison['fecha'], 
           range(len(outliers_comparison)),
           c=outliers_comparison['Consenso_3+'].astype(int),
           cmap='RdYlBu_r', s=1, alpha=0.5)

# Marcar outliers por consenso
outlier_dates = outliers_comparison[outliers_comparison['Consenso_3+']]['fecha']
outlier_indices = outliers_comparison[outliers_comparison['Consenso_3+']].index

ax.scatter(outlier_dates, outlier_indices, 
           c='red', s=20, alpha=0.7, marker='x', label='Outlier (Consenso 3+)')

ax.set_xlabel('Fecha')
ax.set_ylabel('Índice de Observación')
ax.set_title(f'Distribución Temporal de Outliers (Consenso 3+ métodos)\nTotal: {len(outlier_dates)} outliers')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPrimeros 10 outliers detectados por consenso:")
print(outliers_comparison[outliers_comparison['Consenso_3+']][['fecha']].head(10).to_string(index=True))

In [ ]:
# Análisis de las variables originales en outliers detectados por consenso
outlier_indices_consensus = outliers_comparison[outliers_comparison['Consenso_3+']].index
normal_indices_consensus = outliers_comparison[~outliers_comparison['Consenso_3+']].index

# Comparar estadísticas entre outliers y normales
print("\nCOMPARACIÓN DE VARIABLES: OUTLIERS vs NORMALES (Consenso 3+)")
print("=" * 80)

comparison_stats = pd.DataFrame({
    'Normal_Media': df_analysis.loc[normal_indices_consensus].mean(),
    'Outlier_Media': df_analysis.loc[outlier_indices_consensus].mean(),
    'Normal_Std': df_analysis.loc[normal_indices_consensus].std(),
    'Outlier_Std': df_analysis.loc[outlier_indices_consensus].std(),
    'Diferencia_Media': df_analysis.loc[outlier_indices_consensus].mean() - df_analysis.loc[normal_indices_consensus].mean()
})

comparison_stats['Diferencia_%'] = (comparison_stats['Diferencia_Media'] / comparison_stats['Normal_Media']) * 100

print(comparison_stats.to_string())

In [ ]:
# Boxplots comparativos para variables seleccionadas
# Seleccionar variables con mayor diferencia
top_diff_vars = comparison_stats.sort_values(by='Diferencia_%', key=abs, ascending=False).head(8).index.tolist()

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, var in enumerate(top_diff_vars):
    data_to_plot = [
        df_analysis.loc[normal_indices_consensus, var],
        df_analysis.loc[outlier_indices_consensus, var]
    ]
    
    bp = axes[idx].boxplot(data_to_plot, labels=['Normal', 'Outlier'],
                           patch_artist=True, showmeans=True)
    
    # Colorear cajas
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][1].set_facecolor('lightcoral')
    
    axes[idx].set_title(f'{var}\n(Δ: {comparison_stats.loc[var, "Diferencia_%"]:.1f}%)', fontsize=9)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Comparación de Variables: Outliers vs Normales (Top 8 diferencias)', fontsize=14, y=1.001)
plt.tight_layout()
plt.show()

## 7. Análisis de Outliers Específicos

In [ ]:
# Examinar los outliers más extremos
print("\nOUTLIERS MÁS EXTREMOS (Top 10)")
print("=" * 80)

# Usar distancia PCA como medida de extremidad
df_with_pca_dist = df.copy()
df_with_pca_dist['PCA_Distance'] = pca_distances
df_with_pca_dist['Is_Outlier_Consensus'] = outliers_comparison['Consenso_3+'].values

# Top 10 outliers más extremos
top_outliers = df_with_pca_dist[df_with_pca_dist['Is_Outlier_Consensus']].nlargest(10, 'PCA_Distance')

print("\nTop 10 outliers por distancia PCA:")
print(top_outliers[['fecha', 'PCA_Distance', 'precipitacion_mm', 'temp_media_c', 
                     'evapotranspiracion_mm', 'humedad_relativa_pct']].to_string(index=True))

In [ ]:
# Analizar un outlier específico en detalle
if len(top_outliers) > 0:
    # Seleccionar el outlier más extremo
    extreme_outlier_idx = top_outliers.index[0]
    extreme_outlier_date = top_outliers.iloc[0]['fecha']
    
    print(f"\nANÁLISIS DETALLADO DEL OUTLIER MÁS EXTREMO")
    print("=" * 80)
    print(f"Fecha: {extreme_outlier_date}")
    print(f"Índice: {extreme_outlier_idx}")
    print(f"Distancia PCA: {pca_distances[extreme_outlier_idx]:.4f}")
    
    print("\nValores de las variables:")
    print(df_analysis.loc[extreme_outlier_idx].to_string())
    
    print("\nZ-Scores de las variables:")
    z_scores_outlier = (df_analysis.loc[extreme_outlier_idx] - df_analysis.mean()) / df_analysis.std()
    z_scores_sorted = z_scores_outlier.sort_values(key=abs, ascending=False)
    print(z_scores_sorted.head(10).to_string())
    
    # Visualizar Z-scores
    plt.figure(figsize=(12, 6))
    plt.barh(range(len(z_scores_sorted)), z_scores_sorted.values,
             color=['red' if abs(x) > 3 else 'blue' for x in z_scores_sorted.values],
             alpha=0.7, edgecolor='black')
    plt.yticks(range(len(z_scores_sorted)), z_scores_sorted.index)
    plt.xlabel('Z-Score')
    plt.title(f'Z-Scores del Outlier más Extremo ({extreme_outlier_date})')
    plt.axvline(x=3, color='red', linestyle='--', alpha=0.5, label='Umbral +3σ')
    plt.axvline(x=-3, color='red', linestyle='--', alpha=0.5, label='Umbral -3σ')
    plt.axvline(x=0, color='black', linewidth=1)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 8. Conclusiones y Recomendaciones

### 8.1. Resumen de Resultados PCA

**Análisis de Componentes Principales:**
- Se aplicó PCA a 18 variables hidrológicas del dataset (1940-2024)
- Las primeras componentes principales capturan la mayoría de la varianza:
  - PC1: [Ver resultado en celda anterior]% de varianza
  - PC2: [Ver resultado en celda anterior]% de varianza
  - PC3: [Ver resultado en celda anterior]% de varianza
- Para capturar el 90% de la varianza, se requieren aproximadamente [Ver resultado] componentes

**Interpretación de Componentes:**
- Las componentes principales revelan patrones de covariación entre variables climáticas
- Variables altamente correlacionadas (ej. temperaturas, humedades del suelo) se agrupan en componentes comunes
- La reducción dimensional es efectiva, permitiendo representar datos multidimensionales en 2-3 dimensiones

### 8.2. Resumen de Detección de Outliers

**Métodos Aplicados:**
1. **Z-Score:** Detección univariada clásica (umbral 3σ)
2. **Isolation Forest:** Algoritmo basado en árboles de decisión
3. **Local Outlier Factor (LOF):** Densidad local de vecinos
4. **Elliptic Envelope:** Asume distribución Gaussiana multivariada
5. **PCA Distance:** Distancia euclidiana en espacio PCA reducido

**Resultados:**
- Los diferentes métodos detectan entre ~5% de outliers (configurado)
- El **consenso (3+ métodos)** identifica outliers más robustos y confiables
- Los outliers detectados muestran valores extremos principalmente en:
  - [Variables con mayor diferencia - ver análisis previo]

### 8.3. Recomendaciones

**Para Análisis Futuros:**
1. **Investigar outliers por consenso:** Los días identificados por múltiples métodos requieren atención especial
2. **Validación temporal:** Verificar si outliers corresponden a eventos meteorológicos extremos conocidos
3. **Análisis estacional:** Repetir detección de outliers por estación del año
4. **Imputación vs Exclusión:** Decidir si outliers deben ser:
   - Excluidos del análisis
   - Transformados/imputados
   - Analizados por separado como eventos extremos

**Para Modelado:**
1. **Uso de PCA:** Considerar usar componentes principales en lugar de variables originales para:
   - Reducir multicolinealidad
   - Simplificar modelos
   - Mejorar interpretabilidad
2. **Tratamiento de outliers:** Evaluar modelos robustos a outliers o pre-procesamiento específico
3. **Segmentación:** Considerar análisis separados para periodos normales vs extremos

**Limitaciones:**
1. El análisis excluye la variable `almacenamiento_hm3` por valores faltantes
2. Los outliers multivariados pueden no ser outliers en variables individuales
3. La configuración de umbral de contaminación (5%) es arbitraria y puede ajustarse

In [ ]:
# Guardar resultados para análisis posterior
print("Guardando resultados...")

# DataFrame con componentes principales
df_pca_full = df_pca.copy()
df_pca_full.to_csv('../data/processed/pca_components.csv', index=False)
print("  ✓ Componentes PCA guardadas en: data/processed/pca_components.csv")

# DataFrame con detección de outliers
outliers_comparison.to_csv('../data/processed/outliers_detection.csv', index=False)
print("  ✓ Detección de outliers guardada en: data/processed/outliers_detection.csv")

# Guardar loadings
loadings_df.to_csv('../data/processed/pca_loadings.csv', index=True)
print("  ✓ Loadings PCA guardados en: data/processed/pca_loadings.csv")

print("\n¡Análisis completado exitosamente!")

In [ ]:
# Resumen final
print("\n" + "=" * 80)
print("RESUMEN FINAL DEL ANÁLISIS")
print("=" * 80)
print(f"\nDataset analizado: datos_hidrologicos_completos.csv")
print(f"Periodo: {df['fecha'].min()} a {df['fecha'].max()}")
print(f"Total de observaciones: {len(df):,}")
print(f"Variables analizadas: {len(numeric_cols)}")
print(f"\nPCA:")
print(f"  - Componentes óptimas (90% varianza): {optimal_components}")
print(f"  - Reducción dimensional: {len(numeric_cols)} → {optimal_components} ({(1-optimal_components/len(numeric_cols))*100:.1f}% reducción)")
print(f"\nOutliers (Consenso 3+ métodos):")
print(f"  - Total detectados: {outliers_comparison['Consenso_3+'].sum():,} ({outliers_comparison['Consenso_3+'].sum()/len(df)*100:.2f}%)")
print(f"  - Observaciones normales: {(~outliers_comparison['Consenso_3+']).sum():,} ({(~outliers_comparison['Consenso_3+']).sum()/len(df)*100:.2f}%)")
print(f"\nArchivos generados:")
print(f"  - pca_components.csv")
print(f"  - outliers_detection.csv")
print(f"  - pca_loadings.csv")
print("\n" + "=" * 80)